<a href="https://colab.research.google.com/github/novalsunn123/finetune/blob/main/finetuning2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.4: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.6.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:
# 3. Hàm xem dataset
def view_dataset(dataset, name="Dataset", num_samples=2):
    print(f"\n--- Xem {name} ---")
    print(f"Số bản ghi: {len(dataset)}")
    print(f"Các cột: {dataset.column_names}")
    print(f"Mẫu {num_samples} bản ghi đầu tiên:")
    for i in range(min(num_samples, len(dataset))):
        print(f"Bản ghi {i}:")
        print(dataset[i])
    print("-------------------")

In [ ]:
from datasets import Dataset
print("\nTải file security_dataset.json...")
dataset_path = '/content/security_dataset.json'

# Kiểm tra file tồn tại, nếu không thì yêu cầu tải lên
if not os.path.exists(dataset_path):
    print(f"File '{dataset_path}' không tồn tại. Tải file lên...")
    uploaded = files.upload()
    if 'security_dataset.json' not in uploaded:
        print("Error: Vui lòng tải file 'security_dataset.json'")
        raise FileNotFoundError("File 'security_dataset.json' không được tải lên.")

try:
    with open(dataset_path, 'r') as f:
        data = json.load(f)

    # Chuyển thành Dataset để kiểm tra
    raw_dataset = Dataset.from_list(data)
    view_dataset(raw_dataset, "Raw Dataset")

except FileNotFoundError:
    print(f"Error: File '{dataset_path}' not found.")
    raise
except json.JSONDecodeError:
    print("Error: Invalid JSON format in the file.")
    raise



Tải file security_dataset.json...

--- Xem Raw Dataset ---
Số bản ghi: 37
Các cột: ['input', 'output']
Mẫu 2 bản ghi đầu tiên:
Bản ghi 0:
{'input': '{"eventData_httpHostName":"None","eventData_httpMethod":"None","eventData_matchedSample":"None","eventData_sourceIP":"192.168.136.158","eventData_waapIncidentType":"None","eventName":"Check Point Nano-service started","eventPriority":"Medium","eventSeverity":"Info","eventType":"Event Driven","logline":"Check Point Nano-service started","num_models_detected":2,"timestamp":"Fri, 20 Jun 2025 14:04:40 GMT"}', 'output': {'attack_type': 'No Attack', 'source_ip': '192.168.136.158'}}
Bản ghi 1:
{'input': '{"eventData_httpHostName":"None","eventData_httpMethod":"None","eventData_matchedSample":"None","eventData_sourceIP":"192.168.136.158","eventData_waapIncidentType":"None","eventName":"Check Point Nano-service started","eventPriority":"Medium","eventSeverity":"Info","eventType":"Event Driven","logline":"Check Point Nano-service started","num_mode

dataset phân tích log openappsec

In [ ]:
def generate_conversation(examples):
    conversations = []
    for input_data, output_data in zip(examples["input"], examples["output"]):
        attack_type = output_data["attack_type"]
        source_ip = output_data["source_ip"]

        conversation = [
            {
                "role": "user",
                "content": f"{input_data}\nWhat is the attack type and source IP?"
            },
            {
                "role": "assistant",
                "content": f"Attack type: {attack_type}, Source IP: {source_ip}"
            }
        ]
        conversations.append(conversation)
    return {"conversations": conversations}


In [ ]:
print("\nTạo conversation...")
conversation_dataset = raw_dataset.map(generate_conversation, batched=True)
view_dataset(conversation_dataset, "Conversation Dataset")


Tạo conversation...


Map:   0%|          | 0/37 [00:00<?, ? examples/s]


--- Xem Conversation Dataset ---
Số bản ghi: 37
Các cột: ['input', 'output', 'conversations']
Mẫu 2 bản ghi đầu tiên:
Bản ghi 0:
{'input': '{"eventData_httpHostName":"None","eventData_httpMethod":"None","eventData_matchedSample":"None","eventData_sourceIP":"192.168.136.158","eventData_waapIncidentType":"None","eventName":"Check Point Nano-service started","eventPriority":"Medium","eventSeverity":"Info","eventType":"Event Driven","logline":"Check Point Nano-service started","num_models_detected":2,"timestamp":"Fri, 20 Jun 2025 14:04:40 GMT"}', 'output': {'attack_type': 'No Attack', 'source_ip': '192.168.136.158'}, 'conversations': [{'content': '{"eventData_httpHostName":"None","eventData_httpMethod":"None","eventData_matchedSample":"None","eventData_sourceIP":"192.168.136.158","eventData_waapIncidentType":"None","eventName":"Check Point Nano-service started","eventPriority":"Medium","eventSeverity":"Info","eventType":"Event Driven","logline":"Check Point Nano-service started","num_mode

In [ ]:
# 7. Áp dụng chat template (sửa lỗi thẻ <think> thừa)
print("\nÁp dụng chat template...")
reasoning_conversations = []
for conv in conversation_dataset["conversations"]:
    text = tokenizer.apply_chat_template(
        conv,
        tokenize=False,
        add_generation_prompt=False,  # Tắt để tránh prompt thừa
        enable_thinking=False,
    )
    # Loại bỏ thẻ <think> nếu có
    if "<think>" in text:
        text = text.replace("<think>\n\n</think>\n\n", "")
    reasoning_conversations.append(text)


Áp dụng chat template...


In [ ]:
# Kiểm tra reasoning_conversations
print("\nKiểm tra reasoning_conversations:")
for i in range(min(2, len(reasoning_conversations))):
    print(f"Chuỗi {i}:")
    print(reasoning_conversations[i])


Kiểm tra reasoning_conversations:
Chuỗi 0:
<|im_start|>user
{"eventData_httpHostName":"None","eventData_httpMethod":"None","eventData_matchedSample":"None","eventData_sourceIP":"192.168.136.158","eventData_waapIncidentType":"None","eventName":"Check Point Nano-service started","eventPriority":"Medium","eventSeverity":"Info","eventType":"Event Driven","logline":"Check Point Nano-service started","num_models_detected":2,"timestamp":"Fri, 20 Jun 2025 14:04:40 GMT"}
What is the attack type and source IP?<|im_end|>
<|im_start|>assistant
Attack type: No Attack, Source IP: 192.168.136.158<|im_end|>

Chuỗi 1:
<|im_start|>user
{"eventData_httpHostName":"None","eventData_httpMethod":"None","eventData_matchedSample":"None","eventData_sourceIP":"192.168.136.158","eventData_waapIncidentType":"None","eventName":"Check Point Nano-service started","eventPriority":"Medium","eventSeverity":"Info","eventType":"Event Driven","logline":"Check Point Nano-service started","num_models_detected":2,"timestamp"

In [ ]:
import pandas as pd
from datasets import Dataset
# 8. Chuyển thành Dataset cuối cùng
print("\nTạo Dataset cuối cùng...")
data_series = pd.Series(reasoning_conversations)
data_series.name = "text"
combined_dataset = Dataset.from_pandas(pd.DataFrame(data_series))
combined_dataset = combined_dataset.shuffle(seed=3407)
view_dataset(combined_dataset, "Final Dataset")


Tạo Dataset cuối cùng...

--- Xem Final Dataset ---
Số bản ghi: 37
Các cột: ['text']
Mẫu 2 bản ghi đầu tiên:
Bản ghi 0:
{'text': '<|im_start|>user\n{"eventData_httpHostName":"192.168.136.128","eventData_httpMethod":"POST","eventData_matchedSample":"<script>alert(\'xss test\')</script>","eventData_sourceIP":"192.168.136.1","eventData_waapIncidentType":"Cross Site Scripting","eventName":"Web Request","eventPriority":"High","eventSeverity":"Critical","eventType":"Event Driven","logline":"Web Request","num_models_detected":2,"timestamp":"Fri, 20 Jun 2025 15:25:17 GMT"}\nWhat is the attack type and source IP?<|im_end|>\n<|im_start|>assistant\nAttack type: Cross Site Scripting, Source IP: 192.168.136.1<|im_end|>\n'}
Bản ghi 1:
{'text': '<|im_start|>user\n{"eventData_httpHostName":"192.168.136.128","eventData_httpMethod":"POST","eventData_matchedSample":"\\"\' or \'1\'=\'1\' --\\",","eventData_sourceIP":"192.168.136.1","eventData_waapIncidentType":"SQL Injection","eventName":"Web Request","e

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,  # Đảm bảo model đã được tải (ví dụ: Qwen3-4B)
    tokenizer=tokenizer,  # Đảm bảo tokenizer đã được tải
    train_dataset=combined_dataset,  # Dataset đã chuẩn bị trước đó
    eval_dataset=None,
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=30,
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
        output_dir="/content/qwen3-4b-finetuned-security",  # Thư mục lưu mô hình sau huấn luyện
        save_strategy="steps",  # Lưu checkpoint sau mỗi số bước
        save_steps=10,  # Lưu sau mỗi 10 bước (vì max_steps=30, sẽ có 3 checkpoint)
    ),
)

# Bắt đầu huấn luyện
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/37 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 37 | Num Epochs = 6 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288/4,000,000,000 (1.65% trained)


Step,Training Loss
1,3.004800
2,3.098000
3,2.942400
4,2.652600
5,2.284400
6,1.972100
7,1.695900
8,1.428400
9,1.157400
10,0.993200


TrainOutput(global_step=30, training_loss=0.8749007719258467, metrics={'train_runtime': 118.8294, 'train_samples_per_second': 2.02, 'train_steps_per_second': 0.252, 'total_flos': 871804194585600.0, 'train_loss': 0.8749007719258467})

In [ ]:
from transformers import TextStreamer
print("\nKiểm tra mô hình với một log sự kiện...")
messages = [
    {
        "role": "user",
        "content": """{"eventData_httpHostName":"192.168.136.128","eventData_httpMethod":"POST","eventData_matchedSample":"<script>alert('xss test')</script>","eventData_sourceIP":"192.168.136.1","eventData_waapIncidentType":"Cross Site Scripting","eventName":"Web Request","eventPriority":"High","eventSeverity":"Critical","eventType":"Event Driven","logline":"Web Request","num_models_detected":2,"timestamp":"Fri, 20 Jun 2025 14:41:29 GMT"}
What is the attack type and source IP?"""
    }
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)

print("Kết quả:")
_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.8,
    top_k=20,
    streamer=TextStreamer(tokenizer, skip_prompt=True),
)


Kiểm tra mô hình với một log sự kiện...
Kết quả:
The attack type is **Cross Site Scripting** and the source IP is **192.168.136.1**.<|im_end|>


In [ ]:
print("\nLưu mô hình vào Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/qwen3-4b-finetuned-security /content/drive/MyDrive/qwen3-4b-finetuned-security
print("Mô hình đã được lưu vào /content/drive/MyDrive/qwen3-4b-finetuned-security")


Lưu mô hình vào Google Drive...
Mounted at /content/drive
Mô hình đã được lưu vào /content/drive/MyDrive/qwen3-4b-finetuned-security


In [ ]:
messages = [
      {"role": "user", "content": "What could go wrong if I write the code differently?"},

]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

```c
#include<stdio.h>
#include<stdlib.h>

int main() {
    int *a = (int*)malloc(10 * sizeof(int));
    if (a == NULL) {
        printf("Memory allocation failed.\n");
        return 1;
    }
    for (int i = 0; i < 10; i++) {
        scanf("%d", &a[i]);
    }
    printf("The sum is %d.\n", a[10]);
    free(a);
    return 0;
}
```
```c
#include<stdio.h>
#include<stdlib.h>

int main() {
    int *a = (int*)malloc(10 * sizeof(int));
    if (a == NULL) {
        printf("Memory allocation failed.\n");
        return 1;
    }
    for (int i = 0; i < 10; i++) {
        scanf("%d", &a[i]);
    }
    printf("The sum is %d.\n", a[10]);
    free(a);
    return 0;
}
```<|im_end|>


In [ ]:
import os

# Đường dẫn đến thư mục
output_dir = "/content/qwen3-4b-finetuned-security"

# Kiểm tra xem thư mục có tồn tại không
if os.path.exists(output_dir):
    print(f"Thư mục {output_dir} tồn tại. Nội dung:")
    # Liệt kê các thư mục con và tệp
    for item in os.listdir(output_dir):
        print(item)
        # Nếu là thư mục con (như checkpoint-10), liệt kê nội dung bên trong
        item_path = os.path.join(output_dir, item)
        if os.path.isdir(item_path):
            print(f"  Nội dung của {item}:")
            for sub_item in os.listdir(item_path):
                print(f"    {sub_item}")
else:
    print(f"Thư mục {output_dir} không tồn tại. Có thể huấn luyện chưa hoàn thành.")

Thư mục /content/qwen3-4b-finetuned-security tồn tại. Nội dung:
checkpoint-30
  Nội dung của checkpoint-30:
    scaler.pt
    vocab.json
    special_tokens_map.json
    merges.txt
    tokenizer_config.json
    scheduler.pt
    tokenizer.json
    optimizer.pt
    adapter_config.json
    trainer_state.json
    rng_state.pth
    README.md
    adapter_model.safetensors
    added_tokens.json
    chat_template.jinja
    training_args.bin
checkpoint-20
  Nội dung của checkpoint-20:
    scaler.pt
    vocab.json
    special_tokens_map.json
    merges.txt
    tokenizer_config.json
    scheduler.pt
    tokenizer.json
    optimizer.pt
    adapter_config.json
    trainer_state.json
    rng_state.pth
    README.md
    adapter_model.safetensors
    added_tokens.json
    chat_template.jinja
    training_args.bin
checkpoint-10
  Nội dung của checkpoint-10:
    scaler.pt
    vocab.json
    special_tokens_map.json
    merges.txt
    tokenizer_config.json
    scheduler.pt
    tokenizer.json
    optimizer.

In [ ]:
# Lưu LoRA adapters từ checkpoint cuối cùng
model.save_pretrained("./qwen3-4b-finetuned/checkpoint-30")  # Local saving
tokenizer.save_pretrained("./qwen3-4b-finetuned/checkpoint-30")
# model.push_to_hub("your_name/qwen3-4b-finetuned", token="...")  # Online saving
# tokenizer.push_to_hub("your_name/qwen3-4b-finetuned", token="...")  # Online saving

# Tải lại LoRA adapters để sử dụng (bật False thành True nếu cần)
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="./qwen3-4b-finetuned/checkpoint-30",  # Đường dẫn đến checkpoint
        max_seq_length=2048,
        load_in_4bit=True,
    )